# SettingWithCopyWarning


- 위와 같은 에러를 많이 보앗을것이다.
- Pandas 를 주로 사용하는 사람이라면 무조건 보았을 에러이지만 어느정도는 돌아가기 때문에 그냥 지나친 사람들도 있을것이다.
- 하지만 저 warning 이 왜 뜨는것인지에 대해서 제대로 알아야할 것이다.

In [23]:
import pandas as pd
import numpy as np
df = pd.DataFrame([-1,1,2,3,4,5], columns=['NUM'])

In [25]:
# df 로부터 새로운 view(파생데이터) 생성
df_pos = df[df.NUM >= 0]
df_pos

,NUM
1,1
2,2
3,3
4,4
5,5


In [26]:
# 파생 데이터에 수정을 가함
df_pos['new'] = df_pos['NUM']*2

C:\Users\goran\Anaconda3\envs\tensor\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
# 수정은 된 모습
df_pos

,NUM,new
1,1,2
2,2,4
3,3,6
4,4,8
5,5,10


**위 에러는 언제 일어날까?**
- 아니 분명히 new columns 에 값도 잘 붙었는데 왜 error 가 뜬 것일까요?
- 위 에러는 원천이 되는 DataFrame의 일부로 구성된 파생 DataFrame에 변경을 가하는 경우 발생합니다.
    - 이 경우에 df_pos 는 df 의 일부로 구성된 파생 Dataframe 입니다. 
    - 그런게 df_pos['new'] 로 col 을 늘리려는(파생 df를 변경) 시도를 해서 에러가 발생하는것입니다!
- 그러면 왜 이런짓을 하면 경고를 띄울까요? 우선 pandas 의 작동원리부터 살펴봅시다.

**pandas 의 수정은 어떻게 이루어질까?**
- DataFrame도 자료구조(data structure)인 셈이므로 이를 표현하기 위해 메모리를 점유합니다. 
- 하지만 이 때 등장하는 모든 DataFrame에 메모리를 할당하려 들면 금방 바닥을 보이게 되겠지요. 
- 그러므로 Pandas에서는 원천 DataFrame을 생성할 때만 메모리를 할당하고 이로부터 파생되는 DataFrame은 원천 DataFrame을 이미 적재된 메모리에 접근케 하며 이로 인해 메모리를 절약합니다.
- 원천 DataFrame의 부분집합(subset)에 view를 형성한다 보면 됩니다.


**❓❓잠깐! 여기서 view란?**
1. 뷰는 사용자에게 접근이 허용된 자료만을 제한적으로 보여주기 위해 하나 이상의 기본 테이블로부터 유도된, 이름을 가지는 가상 테이블이다.

2. 뷰는 저장장치 내에 물리적으로 존재하지 않지만 사용자에게 있는 것처럼 간주된다.

3. 뷰는 데이터 보정작업, 처리과정 시험 등 임시적인 작업을 위한 용도로 활용된다.



**파생된 df 를 수정하면 어떤일이 일어날까?**
- 파생된 DataFrame에 열(columns)을 하나 추가하면 어떻게 될까요?
- 한 DataFrame의 부분집합으로 이뤄진 view에 변경을 가하는 것이 됩니다. 그렇다면 프로그램은 어떻게 해석해야할까요?
     1. 원천 DataFrame에도 그 열을 붙여 주어야한다.
     2. 아니면 파생 DataFrame에만 그 열을 붙여주면 된다.
- 위에 대한 정답은 없습니다. 실제 개발하는 사람의 의도에 따라 원하는 바가 다를 수 밖에 없습니다
- 그러므로 pandas 에서는 error 를 띄우는 것입니다.

**어떻게 하는것이 바람직할까?**
- Pandas에서는 파생 DataFrame에 수정을 가하는 것을 권장하지 않습니다.(그래서 경고를 띄우는 것이죠.) 
- 대신 Pandas는 copy() 메서드를 통해 파생 DataFrame에 독립적인 메모리를 부여한 뒤 여기에 수정을 가하도록 유도하지요. view를 실질화(materialization)시키는 것이죠.
- 컨대 SettingWithCopyWarning의 발생을 예방하기 위한 가장 쉬운 방법은 파생 DataFrame에 copy() 메서드를 실행해, 원천 DataFrame과 별도인 메모리 공간을 파생 DataFrame에 부여하는 것입니다. 원천과 파생이 완전히 갈라서는 것이지요


In [31]:
df = pd.DataFrame([-1,1,2,3,4,5], columns=['NUM'])
# 파생데이터를 생성하지 않고! copy 를 떠서 새로운주소(실체화) 시침
df_pos = df[df.NUM >= 0].copy()
df_pos['new'] = df_pos['NUM']*2
# 이제 에러가 뜨지 않는 모습
df_pos

,NUM,new
1,1,2
2,2,4
3,3,6
4,4,8
5,5,10


# Reference

- https://coding-factory.tistory.com/224
- https://emilkwak.github.io/pandas-dataframe-settingwithcopywarning